In [32]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import json

#directory = 'aixi-models'; objective = 'explored'; y_axis = 'Exploration (%)'
directory = 'aixi-aimu'; objective = 'rewards'; y_axis = 'Average reward'

file = open(directory + '/results-1.json')
data = json.load(file)
file.close()
colors = ['green','red','blue','orange','black','pink','grey']
agents = {'BayesAgent':r'AI$\xi$',
          'Knowledge-seeking agent':'Kullback-Leibler',
          'KullbackLeiblerKSA':'Kullback-Leibler',
          'ShannonKSA':'Shannon',
          'SquareKSA':'Square',
          'Shannon KSA':'Shannon',
          'Square KSA':'Square',
          'ThompsonAgent':'Thompson',
          'QLearn':'Q-Learning',
          'KSA-Dirichlet': 'Kullback-Leibler',
          'Entropy-seeking agent': 'Shannon',
          'Square KSA-Dirichlet': 'Square'}

fig = plt.figure(figsize=(12,8),dpi=200)
for i,k in enumerate(data):
    try:
        d = data[k]
    except KeyError:
        continue
    color = colors[i]
    alpha = 0.2    
    A = np.zeros((d[0]['cycles'],len(d)))
    for j in xrange(len(d)):
        A[:,j] = np.array(d[j][objective])

    mu = np.mean(A,1)
    sigma = np.std(A,1)
    a = np.max(np.vstack((mu-sigma,np.min(A,1))),0)
    b = np.min(np.vstack((sigma+mu,np.array(200*[100]))),0)
    if k in agents:
        k = agents[k]
    plt.plot(a,color=color,alpha=alpha)
    plt.plot(b,color=color,alpha=alpha)
    plt.plot(mu,label=k,color=color,lw=3)
    plt.fill_between(np.arange(len(mu)),a,b,alpha=alpha,color=color)

#plt.title(y_axis,fontsize=25)
plt.xlabel('Cycles',fontsize=25)
plt.ylabel(y_axis,fontsize=25)
plt.legend(loc='upper left',fontsize=25)
plt.savefig(directory + '/plot-paper.png', bbox_inches='tight')
plt.close()

In [27]:
N = 1e3
x = np.linspace(1/N,1,N)
y = -np.log(x)
z = -x

fig = plt.figure(figsize=(10,5),dpi=200)
plt.title(r'Utility functions for Square and Shannon KSA, as a function of $\xi$')
plt.xlabel(r'$\xi$')
plt.ylabel(r'$u(\xi)$')
plt.plot(x,y,label=r'$-log(\xi)$',lw=3)
plt.plot(x,z,label=r'$-\xi$',lw=3)
plt.legend(loc='upper right')
plt.savefig('../../../thesis/figures/square-shannon-utility')
plt.close()